<center> Adversarial attacks <center>



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
%cd '/content/drive/MyDrive/Colab Notebooks/assignment_3'

/content/drive/MyDrive/Colab Notebooks/assignment_3


In [ ]:
import matplotlib.pyplot as plt
import tensorflow.keras as keras
from keras.callbacks import EarlyStopping,ModelCheckpoint
from cifar_utils import *

1- Import the cifar dataset

In [ ]:
x_train, train_labels, x_test, test_labels = load_CIFAR10_data()
classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

170500096/170498071 [==============================] - 21s 0us/step


In [ ]:
#plt.imshow(x_train[0])

2- Train a base model

In [ ]:
input_shape = (32,32,3)
model = build_model_base_CNN(input_shape)

In [ ]:
#model.summary()

In [ ]:
## hot encoding of the labels
y_train = keras.utils.to_categorical(train_labels, 10) 
y_test = keras.utils.to_categorical(test_labels, 10)

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=15)
mc = ModelCheckpoint('best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

history = model.fit(x_train, y_train, epochs=100, batch_size=64,verbose=1,validation_split = 0.2,callbacks=[es,mc])

Epoch 1/100
620/625 [============================>.] - ETA: 0s - loss: 2.0691 - accuracy: 0.2314
Epoch 00001: val_accuracy improved from -inf to 0.32920, saving model to best_model.h5
625/625 [==============================] - 5s 7ms/step - loss: 2.0673 - accuracy: 0.2318 - val_loss: 1.8622 - val_accuracy: 0.3292
Epoch 2/100
621/625 [============================>.] - ETA: 0s - loss: 1.7515 - accuracy: 0.3566
Epoch 00002: val_accuracy improved from 0.32920 to 0.40860, saving model to best_model.h5
625/625 [==============================] - 4s 6ms/step - loss: 1.7507 - accuracy: 0.3572 - val_loss: 1.6233 - val_accuracy: 0.4086
Epoch 3/100
620/625 [============================>.] - ETA: 0s - loss: 1.6138 - accuracy: 0.4071
Epoch 00003: val_accuracy improved from 0.40860 to 0.46440, saving model to best_model.h5
625/625 [==============================] - 4s 6ms/step - loss: 1.6131 - accuracy: 0.4074 - val_loss: 1.4872 - val_accuracy: 0.4644
Epoch 4/100
621/625 [============================

3- Training GAN

In [ ]:
# Idea from : https://machinelearningmastery.com/impressive-applications-of-generative-adversarial-networks/
# Model coming from: https://machinelearningmastery.com/how-to-develop-a-pix2pix-gan-for-image-to-image-translation/

from keras.initializers import RandomNormal
from keras.layers import (Activation,
                          BatchNormalization,
                          Concatenate,
                          Conv2D,
                          Conv2DTranspose,
                          LeakyReLU)
from random import shuffle

# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # add downsampling layer
  g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
  # conditionally add batch normalization
  if batchnorm:
    g = BatchNormalization()(g, training=True)
  # leaky relu activation
  g = LeakyReLU(alpha=0.2)(g)
  return g
 
# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # add upsampling layer
  g = Conv2DTranspose(n_filters, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(layer_in)
  # add batch normalization
  g = BatchNormalization()(g, training=True)
  # conditionally add dropout
  if dropout:
    g = Dropout(0.5)(g, training=True)
  # merge with skip connection
  g = Concatenate()([g, skip_in])
  # relu activation
  g = Activation('relu')(g)
  return g
 
# define the standalone generator model
def define_generator(image_shape=(32,32,3)):
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # image input
  in_image = Input(shape=image_shape)
  # encoder model
  e1 = define_encoder_block(in_image, 512, batchnorm=False)
  e2 = define_encoder_block(e1, 512)
  e3 = define_encoder_block(e2, 512)
  e4 = define_encoder_block(e3, 512)
  # bottleneck, no batch norm and relu
  b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e4)
  b = Activation('relu')(b)
  # decoder model
  d1 = decoder_block(b, e4, 512)
  d2 = decoder_block(d1, e3, 512)
  d3 = decoder_block(d2, e2, 512)
  d4 = decoder_block(d3, e1, 512, dropout=False)
  # output
  g = Conv2DTranspose(3, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d4)
  out_image = Activation('tanh')(g)
  # define model
  model = Model(in_image, out_image)
  return model

In [ ]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
	# generate fake instance
	X = g_model.predict(samples)
	# create 'fake' class labels (0)
	y = zeros((len(X), patch_shape, patch_shape, 1))
	return X, y

In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape = (32,32,3)):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# define the source image
	true_images = Input(shape=image_shape)
	#connect the source image to the generator input
	fake_images = g_model(true_images)
	# connect the source input and generator output to the discriminator input
	d_output = d_model(fake_images)
	fake_labels = tf.math.argmax(d_output)
	true_labels = Input(shape= fake_labels.shape)
	confidence = tf.math.maximum(d_output, 1)
	# src image as input, generated image and classification output
	model = Model(inputs = [true_images, true_labels], outputs = [true_images, fake_images, true_labels, fake_labels, confidence])
	# compile model
	opt = keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss = custom_loss, optimizer=opt)
	return model

In [ ]:
def custom_loss(true_images, fake_images, true_labels, fake_labels, confidence):
  test = true_images-fake_images
  distances = np.array([np.linalg.norm(x) for x in test])
  label_correctness = (true_labels != fake_labels)
  loss = label_correctness*distances + (1-label_correctness)*confidence
  return loss

In [ ]:
def train_gan(d_model, g_model, x_train, y_train, epochs = 20, batch_size = 256):
  for epoch in range(epochs):
    print('Running epoch {}'.format(epoch))
    #Manually enumerate batch
    start_idx = 0
    while len(x_train[start_idx*batch_size:]>=batch_size):
      if start_idx%10 ==0:
        print(start_idx)
      true_images = x_train[start_idx*batch_size:(start_idx+1)*batch_size]
      true_labels = y_train[start_idx:start_idx+batch_size]
      fake_images = g_model.predict(true_images)
      #d_output = model.predict(fake_images)
      #Train model
      g_loss, _, _ = gan_model.train_on_batch([true_images, true_labels], true_labels)
      start_idx +=1
    #generate last fake images
    true_images = x_train[start_idx:]
    true_labels = y_train[start_idx:]
    #fake_images = g_model.predict(true_images)
    #d_output = model.predict(fake_images)
    #fake_labels = d_output.argmax()
    #confidence = np.max(d_output, axis = 1)
    #Calculate loss
    g_loss, _, _ = gan_model.train_on_batch([true_images, true_labels], true_labels)

    print('Shuffling training set')
    random_idx = np.arange(len(x_train))
    shuffle(random_idx)
    x_train = x_train[random_idx]
    y_train = y_train[random_idx]
    start_idx = 0


In [ ]:
# define the models
d_model = model
g_model = define_generator()
# define the composite model
gan_model = define_gan(g_model, d_model)
# train model
train_gan(d_model, g_model, x_train, y_train)

Running epoch 0
0


TypeError: ignored

4- Evaluate on train and test set 

In [ ]:
print("Base accuracy on regular train images:", model.evaluate(x=x_train, y=y_train, verbose=0)[1])

In [ ]:
print("Base accuracy on regular test images:", model.evaluate(x=x_test, y=y_test, verbose=0)[1])

4 - Generate FGSM ATTACK

Example on one image



In [ ]:
image = x_train[1,:]
image_label = y_train[1,:]

In [ ]:
perturbations = adversarial_pattern(model, image.reshape((-1,32,32,3)), image_label).numpy()

In [ ]:
adversarial = image + perturbations * 0.1

In [ ]:
fig=plt.figure(figsize=(10,5))
subplot = fig.add_subplot(1,2,1)
plt.imshow(image.reshape(32,32,3))
subplot.title.set_text("An original image")
subplot = fig.add_subplot(1,2,2)
plt.imshow(adversarial.reshape(32,32,3))
subplot.title.set_text("An FGSM attack")

In [ ]:
print('The predicted class of the original image ===>',classes[model.predict(image.reshape(-1,32,32,3)).argmax()])
print('The predicted class of the adversarial image===>', classes[model.predict(adversarial).argmax()])

Compute the accuracy of the model on adversarial examples only :

In [ ]:
accuracy_FGSM = 0
for i in range(x_test.shape[0]) : 
  image = x_test[i]
  image_label = y_test[i].argmax()
  perturbations = adversarial_pattern(model, image.reshape((-1,32,32,3)), image_label).numpy()
  adversarial = image + perturbations * 0.1
  predicted = model.predict(adversarial).argmax()
  if predicted == image_label : 
    accuracy_FGSM+=1
accuracy_FGSM = accuracy_FGSM/x_test.shape[0]

In [ ]:
accuracy_FGSM

'The accuracy on test set dropped from approximately 81.2% to 48.7%